# Мурашов Леонид Б05-823

fake_job_postings

best on:  
   
SVC  
acc: 0.9723713646532439  
f1:  0.6990288241438318

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('fake_job_postings.csv')

In [2]:
class_weights = dict(df['fraudulent'].value_counts())
print(class_weights)

{0: 17014, 1: 866}


In [3]:
df['location'] = df['location'].astype(str)

In [4]:
text_cols = ['title', 'location', 'company_profile', 'description', 'requirements', 'benefits']
for col in text_cols:
    df[col] = df[col].astype(str)

In [5]:
y_train = df['fraudulent']
df = df.drop(columns=['fraudulent'])

In [6]:
df

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,nan,0,1,0,Other,Internship,NaN,NaN,Marketing
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,nan,0,1,0,NaN,NaN,NaN,NaN,NaN
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,nan,0,0,0,Full-time,NaN,NaN,NaN,NaN
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,nan,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design


## Convert some to categorical onehot

In [7]:
# df['location_cropped'] = df['location'].astype(str).apply(lambda x: x[:2])
# df['location_cropped'].nunique()

In [8]:
df.nunique()

job_id                 17880
title                  11231
location                3106
department              1337
salary_range             874
company_profile         1710
description            14802
requirements           11969
benefits                6206
telecommuting              2
has_company_logo           2
has_questions              2
employment_type            5
required_experience        7
required_education        13
industry                 131
function                  37
dtype: int64

In [9]:
def salary_converter(x, index=0):
    if not isinstance(x, str):
        return np.NaN
    try:
        return int(x.split('-')[index])
    except:
        return
    
df['min_salary'] = df.salary_range.apply(lambda x: salary_converter(x, 0))
df['max_salary'] = df.salary_range.apply(lambda x: salary_converter(x, 1))

df['min_salary'].fillna(df['min_salary'].median(), inplace=True)
df['max_salary'].fillna(df['max_salary'].median(), inplace=True)

# Let's generate many features and pipe it into logreg

### Convert some attributes to categorical

In [10]:
cols_to_categorical = ['employment_type', 'required_experience', 'required_education']
for col in cols_to_categorical:
    df = pd.concat([df, pd.get_dummies(df[col])], axis=1)
df = df.drop(columns=cols_to_categorical)

In [11]:
df = df.fillna(0)

In [12]:
df['delme'] = ''
for col in text_cols:
    df['delme'] += ' ' + df[col]
texts = df['delme']
df = df.drop(columns=['delme'])

In [13]:
df = df.drop(columns=['title','location','department','company_profile','description','requirements',
                     'benefits','industry','function', 'salary_range', 'job_id'])

### Add text mean vector (giant leap in score)

In [14]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

unable to import 'smart_open.gcs', disabling that module


In [15]:
from nltk import word_tokenize

def text_to_vector(text):
    vectors = np.empty((0, 300))
    for word in word_tokenize(text):
        try: 
            vectors = np.vstack([vectors, wv[word]])
        except:
            pass
    if vectors.size == 0:
        return None
    return vectors.mean(axis=0)

texts_vectors = np.zeros((len(df), 300))
from tqdm.notebook import tqdm
for i, text in enumerate(tqdm(texts)):
    texts_vectors[i] = text_to_vector(text)

In [21]:
#np.save('vectors.npy', texts_vectors)

In [24]:
df = pd.concat([df, pd.DataFrame(texts_vectors)], axis=1)

### Add tf-idf ngrams features; smash them via PCA

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,3))
transformed = vectorizer.fit_transform(tqdm(texts))


In [39]:
transformed.shape

(17880, 2965023)

In [45]:
from sklearn.decomposition import TruncatedSVD 
pcaed = TruncatedSVD(100).fit_transform(transformed)

In [48]:
# np.save('pcaed.npy', pcaed)

In [49]:
df = pd.concat([df, pd.DataFrame(pcaed)], axis=1)

# Normalize features

In [50]:
df = (df - df.mean(axis=0)) / df.std(axis=0)

## Evaluation

#### Random baseline


In [52]:
from sklearn.metrics import accuracy_score, f1_score
def evaluate(y_val, y_pred):
    print('acc:', accuracy_score(y_val, y_pred))
    print('f1: ', f1_score(y_val, y_pred))

In [53]:
y_pred = np.random.randint(0, 2, size=(len(y_train),))
print('Random')
evaluate(y_train, y_pred)
print()

y_pred = [0]*len(y_train)
print('Only zeros')
evaluate(y_train, y_pred)

Random
acc: 0.4987136465324385
f1:  0.08419331766629203

Only zeros
acc: 0.9515659955257271
f1:  0.0


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Various models

#### Our models on cross-validation

In [54]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

x_train = df

In [55]:
def train_and_evaluate(model):
    scores = cross_validate(model, x_train, y_train, cv=2, scoring=['accuracy', 'f1'])
    print(type(model).__name__)
    print('acc:', scores['test_accuracy'].mean())
    print('f1: ', scores['test_f1'].mean())
    print()    

model = LogisticRegression(random_state=0, solver='lbfgs', class_weight='balanced')
train_and_evaluate(model) 

model = SVC(random_state=0, class_weight='balanced', gamma='auto')
train_and_evaluate(model)



C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression
acc: 0.9332214765100671
f1:  0.5159526658689004

SVC
acc: 0.9723713646532439
f1:  0.6990288241438318



In [56]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


In [57]:

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5, class_weight='balanced'),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs=12, class_weight='balanced'),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

for model in classifiers:
    train_and_evaluate(model)

DecisionTreeClassifier
acc: 0.8431767337807606
f1:  0.316946357089139

RandomForestClassifier
acc: 0.8915548098434005
f1:  0.3172116260207103

MLPClassifier
acc: 0.9721476510067114
f1:  0.6682609742396277

AdaBoostClassifier
acc: 0.9618008948545862
f1:  0.542826760867998

GaussianNB
acc: 0.579082774049217
f1:  0.17559925262619985



C:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis
acc: 0.9557046979865772
f1:  0.15703800299749607



### Test boosting

In [61]:
import lightgbm as lgb

lgb_train = lgb.Dataset(x_train, y_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20)

y_pred = gbm.predict(x_train, num_iteration=gbm.best_iteration)


In [64]:
f1_score(y_train, np.round(y_pred))

0.5913891145410236